In [3]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud, STOPWORDS 
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# RECOMMENDATION SYSTEM USING COSINE SIMILARITY

In [4]:
imdb_movies = pd.read_csv('movie.csv')

imdb_movies.head()

,userId,Movie_id,Movie_Title,Released_Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,1,1,Prem Sanyas,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,3,3,Shiraz,-2019.0,109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,5,5,Prapancha Pash,-2021.0,90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,6,6,Alam Ara,-2019.0,110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,8,8,Achhut Kanya,-2010.0,105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [5]:
data_recsys=imdb_movies[['Movie_id', 'Movie_Title', 'Released_Year', 'Duration',
                         'Genre','Rating',
                         'Votes','Director','Actor 1','Actor 2','Actor 3'
                         ]].reset_index(drop = True)
data_recsys.head()

,Movie_id,Movie_Title,Released_Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,1,Prem Sanyas,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,3,Shiraz,-2019.0,109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,5,Prapancha Pash,-2021.0,90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,6,Alam Ara,-2019.0,110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,8,Achhut Kanya,-2010.0,105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [6]:
data_recsys.set_index('Movie_id', inplace = True)

data_recsys['Movie_Title'] = data_recsys['Movie_Title'].fillna('').astype('str').str.lower()
data_recsys['Movie_Title'] = data_recsys['Movie_Title'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Released_Year'] = data_recsys['Released_Year'].fillna('').astype('str').str.lower()
data_recsys['Released_Year'] = data_recsys['Released_Year'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Duration'] = data_recsys['Duration'].fillna('').astype('str').str.lower()
data_recsys['Duration'] = data_recsys['Duration'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Genre'] = data_recsys['Genre'].fillna('').astype('str').str.lower()
data_recsys['Genre'] = data_recsys['Genre'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Rating'] = data_recsys['Rating'].fillna('').astype('str').str.lower()
data_recsys['Rating'] = data_recsys['Rating'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Votes'] = data_recsys['Votes'].fillna('').astype('str').str.lower()
data_recsys['Votes'] = data_recsys['Votes'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Director'] = data_recsys['Director'].fillna('').astype('str').str.lower()
data_recsys['Director'] = data_recsys['Director'].str.translate(str.maketrans('', '', string.punctuation))

data_recsys['Actor 1'] = data_recsys['Actor 1'].fillna('').astype('str').str.lower()
data_recsys['Actor 1'] = data_recsys['Actor 1'].str.translate(str.maketrans('', '', string.punctuation))


data_recsys['Actor 2'] = data_recsys['Actor 2'].fillna('').astype('str').str.lower()
data_recsys['Actor 2'] = data_recsys['Actor 2'].str.translate(str.maketrans('', '', string.punctuation))


data_recsys['Actor 3'] = data_recsys['Actor 3'].fillna('').astype('str').str.lower()
data_recsys['Actor 3'] = data_recsys['Actor 3'].str.translate(str.maketrans('', '', string.punctuation))


In [7]:
data_recsys['Discription'] = data_recsys['Movie_Title'] +" " " "+ data_recsys['Released_Year'] +" " " "+data_recsys['Director'] +" " " "+ data_recsys['Actor 1'] 

In [8]:
data_recsys['Discription'] = data_recsys['Discription'].fillna('').astype('str').str.lower()
data_recsys['Discription'] = data_recsys['Discription'].str.translate(str.maketrans('', '', string.punctuation))

In [9]:
data_recsys

,Movie_Title,Released_Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Discription
Movie_id,,,,,,,,,,,
1,prem sanyas,,,drama,,,js randhawa,manmauji,birbal,rajendra bhatia,prem sanyas js randhawa manmauji
3,shiraz,20190,109 min,drama,70,8,gaurav bakshi,rasika dugal,vivek ghamande,arvind jangid,shiraz 20190 gaurav bakshi rasika dugal
5,prapancha pash,20210,90 min,drama musical,,,soumyajit majumdar,sayani gupta,plabita borthakur,roy angana,prapancha pash 20210 soumyajit majumdar say...
6,alam ara,20190,110 min,comedy romance,44,35,ovais khan,prateik,ishita raj,siddhant kapoor,alam ara 20190 ovais khan prateik
8,achhut kanya,20100,105 min,drama,,,amol palekar,rajat kapoor,rituparna sengupta,antara mali,achhut kanya 20100 amol palekar rajat kapoor
...,...,...,...,...,...,...,...,...,...,...,...
16218,jessie,19480,,,,,bhim,,,,jessie 19480 bhim
16219,ottam,19350,,action fantasy,,,dwarka khosla,master shiraz,champa,bibi,ottam 19350 dwarka khosla master shiraz
16220,pengalila,19350,,,,,sadiq ansari,,,,pengalila 19350 sadiq ansari


In [10]:
listStopwords = set(stopwords.words('english'))
filtered = []
ps = PorterStemmer() 
for i, text in enumerate(data_recsys['Discription'].str.split()):
    for word in text:

        if word not in listStopwords:

            word_stemmed = ps.stem(word)
            filtered.append(word_stemmed)
    data_recsys['Discription'][i] = filtered
    filtered = []

In [11]:
count = CountVectorizer()
count_matrix = count.fit_transform(data_recsys['Discription'].fillna('').astype('str').str.lower())

In [12]:
chunk_size = 500 
matrix_len = count_matrix.shape[0] 

def similarity_cosine(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=count_matrix[start:end], Y=count_matrix)
cosine_similarity_all = []
i=0

In [13]:
for chunk_start in range(0, matrix_len, chunk_size):
    
    if i == 0: 
        cosine_sim = similarity_cosine(chunk_start, chunk_start+chunk_size)
    
    else :
        cosine_similarity_chunk= similarity_cosine(chunk_start, chunk_start+chunk_size)
        cosine_sim = np.concatenate((cosine_sim.astype(np.float32), cosine_similarity_chunk.astype(np.float32)))
    
    i= 1

In [14]:
index_movies = pd.Series(data_recsys.index)

# Movies Recommendation function
def get_movies(title, cosine_sim = cosine_sim):
    recommended_movies = []
    index_movie_input = index_movies[index_movies == title].index[0]
    score_movies = pd.Series(cosine_sim[index_movie_input]).sort_values(ascending = False)
    top_10_index_movies = list(score_movies.iloc[1:11].index)
    for i in top_10_index_movies:
        recommended_movies.append(imdb_movies['Movie_Title'].iloc[i] + ' (' + str(imdb_movies['Released_Year'].iloc[i]) + ')')
    return recommended_movies

In [15]:
movie_input = str(input("Please enter the name of the movie.\n"))
get_movies(movie_input.capitalize())

IndexError: index 0 is out of bounds for axis 0 with size 0